In [1]:
import os
import numpy as np
import nibabel as nib
from scans import *
from tumor_centers import *
from skimage.segmentation import morphological_chan_vese

In [2]:
def dice_coefficient(segmented, ground_truth):
    intersection = np.logical_and(segmented, ground_truth)
    return (2. * intersection.sum()) / (segmented.sum() + ground_truth.sum())

def select_slice(scan, label, slice_idx):
    return scan[:, :, slice_idx], label[:, :, slice_idx]

def load_data(scan_path, label_path):
    scan = nib.load(scan_path).get_fdata()
    label = nib.load(label_path).get_fdata()
    return scan, label

In [3]:
dataset_path = 'D:\Dosyalar\dataset\Task06_Lung'
images_train_dir = os.path.join(dataset_path, 'imagesTr')
labels_train_dir = os.path.join(dataset_path, 'labelsTr')

In [4]:
global_dscs = []
global_sensitivities = []
global_specificities = []
global_accuracies = []
global_weights = []
global_TP = 0
global_TN = 0
global_FP = 0
global_FN = 0

In [5]:
with open('results.txt', 'w') as results_file:
    for scan_number in INDEXES[-21:]:
        scan_path = os.path.join(images_train_dir, f'lung_{scan_number}.nii.gz')
        label_path = os.path.join(labels_train_dir, f'lung_{scan_number}.nii.gz')
        scan, label = load_data(scan_path, label_path)
        
        slice_indexes = globals()[f'scan_{scan_number}']
        tumor_centers = globals()[f'tumor_centers_{scan_number}']

        for i, slice_idx in enumerate(slice_indexes):
            slice, slice_label = select_slice(scan, label, slice_idx)
            
            center = tumor_centers[i]
            seed_x, seed_y = int(center[0]), int(center[1])

            # Define an initial circular region around the seed point
            r = 10  # radius of the initial region
            initial_level_set = np.zeros_like(slice)
            X, Y = np.meshgrid(np.arange(slice.shape[1]), np.arange(slice.shape[0]))
            initial_level_set[((X - seed_x) ** 2 + (Y - seed_y) ** 2) < r ** 2] = 1

            segmented_mask = morphological_chan_vese(slice, 5, init_level_set=initial_level_set)
            # 5 was 100

            # Calculate TP, TN, FP, FN
            TP = np.sum(np.logical_and(segmented_mask == 1, slice_label == 1))
            TN = np.sum(np.logical_and(segmented_mask == 0, slice_label == 0))
            FP = np.sum(np.logical_and(segmented_mask == 1, slice_label == 0))
            FN = np.sum(np.logical_and(segmented_mask == 0, slice_label == 1))
            
            global_TP += TP
            global_TN += TN
            global_FP += FP
            global_FN += FN

            # Calculate DSC, Sensitivity, Specificity, Accuracy
            dsc = dice_coefficient(segmented_mask, slice_label)
            print(f'Scan {scan_number} - Slice {slice_idx}: {dsc}')
            sensitivity = TP / (TP + FN) if TP + FN > 0 else 0
            specificity = TN / (TN + FP) if TN + FP > 0 else 0
            accuracy = (TP + TN) / (TP + TN + FP + FN) if TP + TN + FP + FN > 0 else 0

            # Append metrics to arrays
            # Append metrics to global arrays
            global_dscs.append(dsc)
            global_sensitivities.append(sensitivity)
            global_specificities.append(specificity)
            global_accuracies.append(accuracy)
            global_weights.append(np.sum(segmented_mask))

Scan 062 - Slice 145: 0.337152209492635
Scan 062 - Slice 146: 0.7076023391812866
Scan 062 - Slice 147: 0.782608695652174
Scan 062 - Slice 148: 0.8025641025641026
Scan 062 - Slice 149: 0.8372615039281706
Scan 062 - Slice 150: 0.8804469273743016
Scan 062 - Slice 151: 0.9416941694169417
Scan 062 - Slice 152: 0.8670634920634921
Scan 062 - Slice 153: 0.9455782312925171
Scan 062 - Slice 154: 0.8754285714285714
Scan 062 - Slice 155: 0.8224513172966781
Scan 062 - Slice 156: 0.6754491017964072
Scan 062 - Slice 157: 0.685064935064935
Scan 062 - Slice 158: 0.7888198757763976
Scan 062 - Slice 159: 0.48079658605974396
Scan 062 - Slice 160: 0.050955414012738856
Scan 062 - Slice 161: 0.02745995423340961
Scan 064 - Slice 119: 0.1520912547528517
Scan 064 - Slice 120: 0.579172610556348
Scan 064 - Slice 121: 0.7680525164113785
Scan 064 - Slice 122: 0.7712230215827338
Scan 064 - Slice 123: 0.7185990338164251
Scan 064 - Slice 124: 0.6453576864535768
Scan 064 - Slice 125: 0.584942084942085
Scan 064 - Slice 

In [11]:
# Calculate overall weighted mean and standard deviation of metrics
overall_weighted_mean_dsc = np.average(global_dscs, weights=global_weights)
overall_mean_dsc = np.mean(global_dscs) 
overall_weighted_std_dsc = np.sqrt(np.average((global_dscs - overall_weighted_mean_dsc) ** 2, weights=global_weights))
overall_weighted_mean_sensitivity = np.average(global_sensitivities, weights=global_weights)
overall_weighted_std_sensitivity = np.sqrt(np.average((global_sensitivities - overall_weighted_mean_sensitivity) ** 2, weights=global_weights))
overall_weighted_mean_specificity = np.average(global_specificities, weights=global_weights)
overall_weighted_std_specificity = np.sqrt(np.average((global_specificities - overall_weighted_mean_specificity) ** 2, weights=global_weights))
overall_weighted_mean_accuracy = np.average(global_accuracies, weights=global_weights)
overall_weighted_std_accuracy = np.sqrt(np.average((global_accuracies - overall_weighted_mean_accuracy) ** 2, weights=global_weights))
# Calculate overall confusion matrix percentages
true_positive_rate = global_TP / (global_TP + global_FN) * 100
true_negative_rate = global_TN / (global_TN + global_FP) * 100
false_positive_rate = global_FP / (global_FP + global_TN) * 100
false_negative_rate = global_FN / (global_FN + global_TP) * 100

In [6]:
overall_mean_dsc = np.mean(global_dscs)
overall_mean_sensitivity = np.mean(global_sensitivities)
overall_mean_specificity = np.mean(global_specificities)
overall_mean_accuracy = np.mean(global_accuracies)
print("Overall Mean DSC:", overall_mean_dsc)
print("Overall Mean Sensitivity:", overall_mean_sensitivity)
print("Overall Mean Specificity:", overall_mean_specificity)
print("Overall Mean Accuracy:", overall_mean_accuracy)


Overall Mean DSC: 0.4765172744724284
Overall Mean Sensitivity: 0.5980689072667922
Overall Mean Specificity: 0.9994783974622513
Overall Mean Accuracy: 0.9974581941645196


In [12]:
# Print or save overall results
print(f'Overall Weighted Mean DSC: {overall_weighted_mean_dsc}, Standard Deviation: {overall_weighted_std_dsc}')
print(f'Overall Mean DSC: {overall_mean_dsc}')
print(f'Overall Weighted Mean Sensitivity: {overall_weighted_mean_sensitivity}, Standard Deviation: {overall_weighted_std_sensitivity}')
print(f'Overall Weighted Mean Specificity: {overall_weighted_mean_specificity}, Standard Deviation: {overall_weighted_std_specificity}')
print(f'Overall Weighted Mean Accuracy: {overall_weighted_mean_accuracy}, Standard Deviation: {overall_weighted_std_accuracy}')

Overall Weighted Mean DSC: 0.3941895850908255, Standard Deviation: 0.2053702657194895
Overall Mean DSC: 0.4296501826930542
Overall Weighted Mean Sensitivity: 0.6217299645534206, Standard Deviation: 0.32467919047558647
Overall Weighted Mean Specificity: 0.9992630405270954, Standard Deviation: 0.0007707061173864033
Overall Weighted Mean Accuracy: 0.9977284345301944, Standard Deviation: 0.0015548305973034661


In [13]:
# Print confusion matrix percentages
print(f'True Positive Rate: {true_positive_rate}%')
print(f'True Negative Rate: {true_negative_rate}%')
print(f'False Positive Rate: {false_positive_rate}%')
print(f'False Negative Rate: {false_negative_rate}%')

True Positive Rate: 37.01227033694735%
True Negative Rate: 99.93849494258511%
False Positive Rate: 0.06150505741488647%
False Negative Rate: 62.98772966305265%
